## Finding Roots of a Non-Linear Equation

Consider the equation: $5e^{-x} +x - 5 = 0$ . Suppose we want to solve it with (absolute) accuracy tolerance $\epsilon = 10^{-6}$.

### Exercise 1
Solve using binary search. How many iterations are required?

In [50]:
import numpy as np


def f(x):
    return 5 * np.exp(-x) + x - 5


# Binary search (x1 and x2 both on either side of 0)
x2 = 10
x1 = 1
eps = 1e-6

if f(x1) * f(x2) > 0:
    raise ValueError("f(x1) and f(x2) have the same sign")

midpoint = 0

iterations = 0

while abs(f(x2) - f(x1)) > eps:
    midpoint = 0.5 * (x1 + x2)
    iterations += 1

    if np.sign(f(midpoint)) == np.sign(f(x1)):
        x1 = midpoint
    else:
        x2 = midpoint

print(f"Final estimate is {f(midpoint)} at x = {midpoint}")
print(f"Took {iterations} tries")

Final estimate is 1.765653800944733e-07 at x = 4.965114414691925
Took 24 tries


### Exercise 2
Solve using relaxation. How many iterations are required?

In [51]:
# Relaxation method


def isolatedf(x):
    return 5 - 5 * np.exp(-x)  # solve for x in the main equation


x = 10

iterations = 0
while abs(f(x)) > eps:
    x = isolatedf(x)
    iterations += 1

print(f"Took {iterations} tries with f(x) being {f(x)} at x={x}")

Took 5 tries with f(x) being 4.8664622553928893e-08 at x=4.965114282167968


### Exercise 3
Solve using Newton's Method. How many iterations are required?

In [52]:
# Newton's Method


def fprime(x):
    # Done analytically
    return -5 * np.exp(-x) + 1


x = 10
iterations = 0

while abs(f(x)) > eps:
    x = x - f(x) / fprime(x)
    iterations += 1

print(f"Took {iterations} tries with f(x) being {f(x)} at x={x}")

Took 3 tries with f(x) being 6.865619184281968e-12 at x=4.96511423175139


### Exercise 4
Solve using Secant Method. How many iterations are required?

In [53]:
# Secant method (don't know derivative)


def secant(x1, x2, func):
    temp = func(x2) - f(x1)
    return temp / (x2 - x1)


x0 = 10
x1 = 7

iterations = 0

while abs(f(x1)) > eps:

    iterations += 1

    x2 = x1 - f(x1) / secant(x1, x0, f)

    x3 = x2 - f(x2) / secant(x1, x2, f)

    x0, x1 = x2, x3

print(f"Took {iterations} tries with f(x) being {f(x1)} at x={x1}")

Took 2 tries with f(x) being 2.5934809855243657e-12 at x=4.965114231746964


## Fun with Circuits

### Physics background

![Newman's Exercise 6.5](rescap.png)

Consider the above circuit. Suppose the voltage $V_+$ is time-varying and sinusoidal of the form $V_+ = x_+ \exp(i \omega t)$ with $x_+$ a constant.

The resistors in the circuit can be treated using Ohm's law.
For the capacitors the charge $Q$ and voltage $V$ across them are related by the capacitor law $Q=CV$, where $C$ is the capacitance.
Differentiating both sides of this expression gives the current $I$ flowing in on one side of the capacitor and out on the other:
\begin{equation}
  I = \frac{dQ}{dt} = C \frac{dV}{dt}.
\end{equation}
Now assume the voltages at the points labeled 1, 2, and 3 are of the form $V_1 = x_1 \exp(i \omega t)$, $V_2 = x_2 \exp(i \omega t)$, and $V_3 = x_3 \exp(i \omega t)$.
If you add up the currents using Kirchoff's law that at a junction the sum of the currents in equals the sum of the currents out, you can find that the constants $x_1$,
$x_2$, and $x_3$ satisfy the equations

\begin{align*}
  \left( {1\over R_1} + {1\over R_4} + i\omega C_1 \right) x_1 - i\omega C_1 x_2
  & = {x_+\over R_1}\,, \\
  - i\omega C_1 x_1 + \left( {1\over R_2} + {1\over R_5} + i\omega C_1 + i\omega C_2 \right) x_2 - i\omega C_2 x_3
                                                                                           & = {x_+\over R_2}\,, \\
  - i\omega C_2 x_2 + \left( {1\over R_3} + {1\over R_6} + i\omega C_2 \right) x_3
  & = {x_+\over R_3}\,.
\end{align*}
This is a linear system of equations for three complex numbers, $x_1$, $x_2$, and $x_3$.

We will be solving the above linear system of equations in the form $A x = b$, where $x$ is the vector $(x_1 x_2 x_3$ and $b$ is the vector composed of the right-hand sides of the equations above.

The following function takes as input the list of resistance values ($R_1$ to $R_6$) and the list of capacitances ($C_1$ and $C_2$), and returns (as numpy.array) the matrix $A$. 

In [54]:
def CircuitMatrix(R, C):
    """I define the matrix A as a function of the one element we turn from a
    resistor to an inductor
    IN: element [complex]: the resistor or inductor
    R_ [float]: list of resistors R1 to R5
    C_ [complex]: list of capacitances C1 and C2
    """
    A = np.empty((3, 3), complex)

    # 1st line of matrix
    A[0, 0] = 1.0 / R[1] + 1.0 / R[4] + C[1]
    A[0, 1] = -C[1]
    A[0, 2] = 0.0

    # 2nd line of matrix
    A[1, 0] = -C[1]
    print(C[2])
    A[1, 1] = 1.0 / R[2] + 1.0 / R[5] + C[1] + C[2]
    A[1, 2] = -C[2]

    # 3rd line of matrix
    A[2, 0] = 0.0
    A[2, 1] = -C[2]
    A[2, 2] = 1.0 / R[3] + 1.0 / R[6] + C[2]

    return A

And the following function takes as input the list of resistance values and the value of $x+$, and returns (as numpy.array) the vector $b$.

In [55]:
def RHS(R, xplus):
    return xplus * np.array([1.0 / R[1], 1.0 / R[2], 1.0 / R[3]], complex)

### Exercise 5

Use Gaussian Elimination with partial pivoting (see the code fragment below) to solve for $x_1$, $x_2$, and $x_3$. Assume the following:

\begin{align*}
  R_1 &= R_3 = R_5 = 1\,\mathrm{k}\Omega, \\
  R_2 &= R_4 = R_6 = 2\,\mathrm{k}\Omega, \\
  C_1 &= 1\,\mu\mathrm{F},\qquad C_2 = 0.5\,\mu\mathrm{F}, \\
  x_+ &= 3\,\mathrm{V},\qquad \omega = 1000~\text{rad/s}.
\end{align*}

Have your program calculate and print, at $t=0$, the amplitudes of the three voltages $|V_1|$, $|V_2|$, and $|V_3|$ and their phases (i.e. the phases of the coefficients $x_1, x_2, x_3$) in degrees.

*Notice that the matrix for this problem has complex elements. You will need to define a complex array to hold it, but your routine should be able to work with real or complex arguments.*

*Hint: the built-in abs() will compute the magnitude, and numpy.angle() will compute the phase of a complex number. You could also use polar and phase from the cmath package.*

In [56]:
import numpy as np


def GaussElim(A_in, v_in, pivot=False):
    """Implement Gaussian Elimination. This should be non-destructive for input
    arrays, so we will copy A and v to temporary variables
    IN:
    A_in [np.array], the matrix to pivot and triangularize
    v_in [np.array], the RHS vector
    pivot [bool, default-False]: user decides if we pivot or not.
    OUT:
    x, the vector solution of A_in x = v_in"""
    # copy A and v to temporary variables using copy command
    A = np.copy(A_in)
    v = np.copy(v_in)
    N = len(v)

    for m in range(N):
        if pivot:  # This is where I modify GaussElim
            # compare the mth element to all other mth elements below
            ZeRow = m
            for mm in range(m + 1, N):
                if abs(A[mm, m]) > abs(A[ZeRow, m]):
                    ZeRow = mm  # I could swap everytime I find a hit, but that
                    # would be a lot of operations. Instead, I just take note
                    # of which row emerges as the winner

            if ZeRow != m:  # now we know if and what to swap
                A[ZeRow, :], A[m, :] = np.copy(A[m, :]), np.copy(A[ZeRow, :])
                v[ZeRow], v[m] = np.copy(v[m]), np.copy(v[ZeRow])

        # Divide by the diagonal element
        div = A[m, m]
        A[m, :] /= div
        v[m] /= div

        # Now subtract from the lower rows
        for i in range(m + 1, N):
            mult = A[i, m]
            A[i, :] -= mult * A[m, :]
            v[i] -= mult * v[m]

    # Backsubstitution
    # create an array of the same type as the input array
    x = np.empty(N, dtype=v.dtype)
    for m in range(N - 1, -1, -1):
        x[m] = v[m]
        for i in range(m + 1, N):
            x[m] -= A[m, i] * x[i]
    return x


def PartialPivot(A_in, v_in):
    """see textbook p. 222)"""
    return GaussElim(A_in, v_in, True)

In [57]:
# Doing the gaussian elimination method, no need for pivot with these values

R = np.array([0, 1, 2, 1, 2, 1, 2], complex) * 1000.0  # kiloohms to ohms
C = np.array([0, 1, 0.5], complex) * 1e-6  # microfarad to farad

# NOTE: THE DEFINITION OF THE FUNCTION TAKES C[1] and C[2] AS THE VALUES
# NEED TO SET C WITH AN EXTRA VALUE AT THE FIRST INDEX. SAME FOR R

xplus = 3  # V
omega = 1000  # rad/s

C *= 1j * omega  # since this is how the matrix function is defined

A = CircuitMatrix(R, C)  # matrix for the system of linear equations
v = RHS(R, xplus)


# Calculation

Voltages = GaussElim(A, v, pivot=False)

amplitudes = np.abs(Voltages)
phases = np.angle(Voltages)

for i in range(3):
    print(f"|Voltage{i+1}| = {amplitudes[i]} --> Phase = {phases[i]}")

0.0005j
|Voltage1| = 1.7014390658777339 --> Phase = -0.09545371433271428
|Voltage2| = 1.4806053465364062 --> Phase = 0.20216879328855675
|Voltage3| = 1.8607693200562132 --> Phase = -0.07268725004282395
